In [1]:
import pandas as pd       
import matplotlib as mat
import matplotlib.pyplot as plt    
import numpy as np
import seaborn as sns

import random
import os
from numpy.random import seed
seed(42)
random.seed(42)

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import glob
import cv2

from tensorflow.random import set_seed


In [2]:
main_path="C:/Users/cool/Desktop/texniti noimosini/spl"

train_path = os.path.join(main_path,"train")
test_path=os.path.join(main_path,"test")


train_normal = glob.glob(train_path+"/NORMAL/*.jpeg")
train_pneumonia = glob.glob(train_path+"/BACTERIA/*.jpeg")
train_VIRUS = glob.glob(train_path+"/VIRUS/*.jpeg")

test_normal = glob.glob(test_path+"/NORMAL/*.jpeg")
test_pneumonia = glob.glob(test_path+"/BACTERIA/*.jpeg")
test_VIRUS = glob.glob(test_path+"/VIRUS/*.jpeg")


In [3]:
train_list = [x for x in train_normal]
train_list.extend([x for x in train_pneumonia])
train_list.extend([x for x in train_VIRUS])

df_train = pd.DataFrame(np.concatenate([['Normal']*len(train_normal) , ['Virus']*len(train_VIRUS) , ['Pneumonia']*len(train_pneumonia)]), columns = ['class'])
df_train['image'] = [x for x in train_list]

test_list = [x for x in test_normal]
test_list.extend([x for x in test_pneumonia])
test_list.extend([x for x in test_VIRUS])

df_test = pd.DataFrame(np.concatenate([['Normal']*len(test_normal) , ['Pneumonia']*len(test_pneumonia), ['Virus']*len(test_VIRUS)]), columns = ['class'])
df_test['image'] = [x for x in test_list]

In [4]:
print(df_test)
print(df_train)

      class                                              image
0    Normal  C:/Users/cool/Desktop/texniti noimosini/spl\te...
1    Normal  C:/Users/cool/Desktop/texniti noimosini/spl\te...
2    Normal  C:/Users/cool/Desktop/texniti noimosini/spl\te...
3    Normal  C:/Users/cool/Desktop/texniti noimosini/spl\te...
4    Normal  C:/Users/cool/Desktop/texniti noimosini/spl\te...
..      ...                                                ...
619   Virus  C:/Users/cool/Desktop/texniti noimosini/spl\te...
620   Virus  C:/Users/cool/Desktop/texniti noimosini/spl\te...
621   Virus  C:/Users/cool/Desktop/texniti noimosini/spl\te...
622   Virus  C:/Users/cool/Desktop/texniti noimosini/spl\te...
623   Virus  C:/Users/cool/Desktop/texniti noimosini/spl\te...

[624 rows x 2 columns]
          class                                              image
0        Normal  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
1        Normal  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
2        Normal  C:

In [5]:
IMG_SIZE = 224
BATCH = 32
SEED = 42

In [24]:
train_df, val_df = train_test_split(df_train, test_size = 0.20, shuffle=True, random_state = SEED, stratify = df_train['class'])
print(train_df.shape)
print(val_df[:20])

(4169, 2)
          class                                              image
3978  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
648      Normal  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
3267  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
4736  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
1263     Normal  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
4838  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
1275     Normal  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
1109     Normal  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
3113  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
3488  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
4276  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
3645  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
4929  Pneumonia  C:/Users/cool/Desktop/texniti noimosini/spl\tr...
1405      Virus  C:/Users/cool/Desktop/texniti noimo

In [7]:
# https://vijayabhaskar96.medium.com/tutorial-on-keras-flow-from-dataframe-1fd4493d237c

train_datagen = ImageDataGenerator(rescale=1/255.,
                                  zoom_range = 0.1,
                                  rotation_range = 0.1,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1)

val_datagen = ImageDataGenerator(rescale=1/255.)

ds_train = train_datagen.flow_from_dataframe(train_df,
                                             x_col = 'image',
                                             y_col = 'class',
                                             target_size = (IMG_SIZE, IMG_SIZE),
                                             class_mode = 'categorical',
                                             batch_size = BATCH,
                                             seed = SEED)

ds_val = val_datagen.flow_from_dataframe(val_df,                                            
                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'categorical',
                                            batch_size = BATCH,
                                            seed = SEED)

ds_test = val_datagen.flow_from_dataframe(df_test,
                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'categorical',
                                            batch_size = 1,
                                            shuffle = False)

Found 4169 validated image filenames belonging to 3 classes.
Found 1043 validated image filenames belonging to 3 classes.
Found 624 validated image filenames belonging to 3 classes.


In [8]:
print(train_df.shape)

(4169, 2)


In [9]:
#Setting callbakcs

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min',
)


In [10]:
def get_model():    
    #Input shape = [width, height, color channels]
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    hidden=inputs
    
    # Block One
    hidden = layers.Conv2D(16, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    # Block Two
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    # Block Three
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    # Block four
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    # Head
    hidden = layers.Flatten()(hidden)
    hidden = layers.Dense(64, activation='relu',kernel_regularizer=regularizers.L2(0.001))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    #Final Layer (Output)
    output = layers.Dense(3, activation='softmax')(hidden)     
    model = keras.Model(inputs,output)    
    return model

In [11]:
from tensorflow.keras import layers
from tensorflow.keras import regularizers

keras.backend.clear_session()

model = get_model()
model.compile(loss='categorical_crossentropy'
              , optimizer = keras.optimizers.Adam(), metrics='accuracy')

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 54, 54, 32)        0     

In [13]:
history = model.fit(ds_train,
          batch_size = BATCH, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 139s 1s/step - loss: 1.0970 - accuracy: 0.4788 - val_loss: 0.9415 - val_accuracy: 0.5743
Epoch 2/50
130/130 [==============================] - 134s 1s/step - loss: 0.8651 - accuracy: 0.5999 - val_loss: 0.6945 - val_accuracy: 0.6903
Epoch 3/50
130/130 [==============================] - 134s 1s/step - loss: 0.7440 - accuracy: 0.6572 - val_loss: 0.6481 - val_accuracy: 0.6999
Epoch 4/50
130/130 [==============================] - 133s 1s/step - loss: 0.7200 - accuracy: 0.6678 - val_loss: 0.6186 - val_accuracy: 0.7009
Epoch 5/50
130/130 [==============================] - 132s 1s/step - loss: 0.6824 - accuracy: 0.6810 - val_loss: 0.6301 - val_accuracy: 0.7028
Epoch 6/50
130/130 [==============================] - 133s 1s/step - loss: 0.6988 - accuracy: 0.6728 - val_loss: 0.6000 - val_accuracy: 0.7085
Epoch 7/50
130/130 [==============================] - 133s 1s/step - loss: 0.6682 - accuracy: 0.6882 - val_loss: 0.6430 - val_accuracy: 0.6942

In [28]:
def showScore():
    score = model.evaluate(ds_test, steps = len(val_df)/BATCH, verbose = 0)
    print('Val loss:', score[0])
    print('Val accuracy:', score[1])
    best_score = max(history.history['accuracy'])
    print('Best score:',best_score)
    

In [22]:
showScore()

Val loss: 0.6689152121543884
Val accuracy: 0.6749760508537292
Test loss: 0.6621912121772766
Test accuracy: 0.690625011920929
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
0.7092828154563904


In [25]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=4,
    mode='min',
)

In [26]:
history = model.fit(ds_train,
          batch_size = BATCH, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 145s 1s/step - loss: 0.6348 - accuracy: 0.7066 - val_loss: 0.5952 - val_accuracy: 0.7105
Epoch 2/50
130/130 [==============================] - 148s 1s/step - loss: 0.6212 - accuracy: 0.7100 - val_loss: 0.6730 - val_accuracy: 0.6884
Epoch 3/50
130/130 [==============================] - 147s 1s/step - loss: 0.6216 - accuracy: 0.7042 - val_loss: 0.5823 - val_accuracy: 0.7210
Epoch 4/50
130/130 [==============================] - 151s 1s/step - loss: 0.6197 - accuracy: 0.7122 - val_loss: 0.5819 - val_accuracy: 0.7162
Epoch 5/50
130/130 [==============================] - 144s 1s/step - loss: 0.6143 - accuracy: 0.7148 - val_loss: 0.5765 - val_accuracy: 0.7200
Epoch 6/50
130/130 [==============================] - 159s 1s/step - loss: 0.6078 - accuracy: 0.7122 - val_loss: 0.5783 - val_accuracy: 0.7277
Epoch 7/50
130/130 [==============================] - 151s 1s/step - loss: 0.5930 - accuracy: 0.7174 - val_loss: 0.5911 - val_accuracy: 0.7325

In [29]:
showScore()

Val loss: 0.7504679560661316
Val accuracy: 0.6969696879386902
Best score: 0.7200767397880554


In [30]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=4,
    mode='min',
)

In [31]:
history = model.fit(ds_train,
          batch_size = BATCH, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 182s 1s/step - loss: 0.6099 - accuracy: 0.7124 - val_loss: 0.5685 - val_accuracy: 0.7315
Epoch 2/50
130/130 [==============================] - 193s 1s/step - loss: 0.5963 - accuracy: 0.7170 - val_loss: 0.5735 - val_accuracy: 0.7239
Epoch 3/50
130/130 [==============================] - 194s 1s/step - loss: 0.5960 - accuracy: 0.7227 - val_loss: 0.5761 - val_accuracy: 0.7172
Epoch 4/50
130/130 [==============================] - 181s 1s/step - loss: 0.5927 - accuracy: 0.7244 - val_loss: 0.5700 - val_accuracy: 0.7181
Epoch 5/50
130/130 [==============================] - 192s 1s/step - loss: 0.5796 - accuracy: 0.7325 - val_loss: 0.5662 - val_accuracy: 0.7277
Epoch 6/50
130/130 [==============================] - 199s 2s/step - loss: 0.6021 - accuracy: 0.7196 - val_loss: 0.5694 - val_accuracy: 0.7229
Epoch 7/50
130/130 [==============================] - 171s 1s/step - loss: 0.5809 - accuracy: 0.7239 - val_loss: 0.5782 - val_accuracy: 0.7430

In [32]:
showScore()

Val loss: 0.20500215888023376
Val accuracy: 0.9696969985961914
Best score: 0.7342288494110107


In [33]:
print(len(train_df)/BATCH)
print(len(val_df)/BATCH)

130.28125
32.59375


In [34]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min',
)


In [35]:
history = model.fit(ds_train,
          batch_size = BATCH, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 156s 1s/step - loss: 0.5711 - accuracy: 0.7333 - val_loss: 0.5630 - val_accuracy: 0.7296
Epoch 2/50
130/130 [==============================] - 160s 1s/step - loss: 0.5804 - accuracy: 0.7330 - val_loss: 0.5794 - val_accuracy: 0.7191
Epoch 3/50
130/130 [==============================] - 138s 1s/step - loss: 0.5691 - accuracy: 0.7304 - val_loss: 0.5637 - val_accuracy: 0.7306
Epoch 4/50
130/130 [==============================] - 135s 1s/step - loss: 0.5841 - accuracy: 0.7275 - val_loss: 0.5584 - val_accuracy: 0.7354
Epoch 5/50
130/130 [==============================] - 170s 1s/step - loss: 0.5783 - accuracy: 0.7347 - val_loss: 0.5629 - val_accuracy: 0.7267
Epoch 6/50
130/130 [==============================] - 169s 1s/step - loss: 0.5739 - accuracy: 0.7361 - val_loss: 0.5669 - val_accuracy: 0.7383
Epoch 7/50
130/130 [==============================] - 158s 1s/step - loss: 0.5670 - accuracy: 0.7429 - val_loss: 0.6098 - val_accuracy: 0.7181

In [36]:
showScore()

Val loss: 0.853921115398407
Val accuracy: 0.6363636255264282
Best score: 0.7428640127182007


In [40]:
def get_model():    
    
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    hidden=inputs
    
    
    hidden = layers.Conv2D(16, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
      
    
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Flatten()(hidden)
    hidden = layers.Dense(64, activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    
    output = layers.Dense(3, activation='softmax')(hidden)     
    model = keras.Model(inputs,output)    
    return model

In [41]:
model = get_model()
model.compile(loss='categorical_crossentropy'
              , optimizer = keras.optimizers.Adam(), metrics='accuracy')

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_9 (Conv2D)           (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 111, 111, 16)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 54, 54, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 52, 52, 64)        1849

In [43]:
history = model.fit(ds_train,
          batch_size = BATCH, 
          epochs = 50,
          validation_data=ds_val,
          callbacks = early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 155s 1s/step - loss: 1.0349 - accuracy: 0.5114 - val_loss: 0.9633 - val_accuracy: 0.5992
Epoch 2/50
130/130 [==============================] - 147s 1s/step - loss: 0.7937 - accuracy: 0.6306 - val_loss: 0.6741 - val_accuracy: 0.6836
Epoch 3/50
130/130 [==============================] - 147s 1s/step - loss: 0.7058 - accuracy: 0.6673 - val_loss: 0.6262 - val_accuracy: 0.7095
Epoch 4/50
130/130 [==============================] - 155s 1s/step - loss: 0.6549 - accuracy: 0.6863 - val_loss: 0.5797 - val_accuracy: 0.7143
Epoch 5/50
130/130 [==============================] - 165s 1s/step - loss: 0.6385 - accuracy: 0.6901 - val_loss: 0.5602 - val_accuracy: 0.7220
Epoch 6/50
130/130 [==============================] - 162s 1s/step - loss: 0.6335 - accuracy: 0.6908 - val_loss: 0.5562 - val_accuracy: 0.7229
Epoch 7/50
130/130 [==============================] - 142s 1s/step - loss: 0.6091 - accuracy: 0.7028 - val_loss: 0.5465 - val_accuracy: 0.7296

In [44]:
showScore()

Val loss: 0.6171468496322632
Val accuracy: 0.7272727489471436
Best score: 0.7263132929801941


In [45]:
def get_model():    
    #Input shape = [width, height, color channels]
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    hidden=inputs
    
    
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    hidden = layers.Dropout(0.2)(hidden)
      
    
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)    
    
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Flatten()(hidden)
    hidden = layers.Dense(64, activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    
    output = layers.Dense(3, activation='softmax')(hidden)     
    model = keras.Model(inputs,output)    
    return model

In [46]:
model = get_model()
model.compile(loss='categorical_crossentropy'
              , optimizer = keras.optimizers.Adam(), metrics='accuracy')

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_14 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 109, 109, 32)      9248      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 54, 54, 32)       0         
 g2D)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 54, 54, 32)        0   

In [48]:
history = model.fit(ds_train,
          batch_size = BATCH, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 181s 1s/step - loss: 1.0703 - accuracy: 0.4845 - val_loss: 1.0622 - val_accuracy: 0.4851
Epoch 2/50
130/130 [==============================] - 165s 1s/step - loss: 1.0120 - accuracy: 0.5243 - val_loss: 0.9018 - val_accuracy: 0.6194
Epoch 3/50
130/130 [==============================] - 166s 1s/step - loss: 0.8002 - accuracy: 0.6239 - val_loss: 0.6847 - val_accuracy: 0.6826
Epoch 4/50
130/130 [==============================] - 165s 1s/step - loss: 0.7113 - accuracy: 0.6651 - val_loss: 0.6498 - val_accuracy: 0.6942
Epoch 5/50
130/130 [==============================] - 163s 1s/step - loss: 0.6827 - accuracy: 0.6719 - val_loss: 0.6056 - val_accuracy: 0.7114
Epoch 6/50
130/130 [==============================] - 163s 1s/step - loss: 0.6564 - accuracy: 0.6812 - val_loss: 0.6101 - val_accuracy: 0.7028
Epoch 7/50
130/130 [==============================] - 163s 1s/step - loss: 0.6456 - accuracy: 0.6831 - val_loss: 0.6085 - val_accuracy: 0.7066

In [49]:
showScore()

Val loss: 0.4177037179470062
Val accuracy: 0.8787878751754761
Best score: 0.7145598530769348


In [55]:
history = model.fit(ds_train,
          batch_size = BATCH, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=100,
          validation_steps=40);

Epoch 1/50
100/100 [==============================] - 145s 1s/step - loss: 0.5829 - accuracy: 0.7170 - val_loss: 0.5668 - val_accuracy: 0.7114
Epoch 2/50
100/100 [==============================] - 118s 1s/step - loss: 0.5697 - accuracy: 0.7199
Epoch 3/50
100/100 [==============================] - 117s 1s/step - loss: 0.5652 - accuracy: 0.7237
Epoch 4/50
100/100 [==============================] - 126s 1s/step - loss: 0.5683 - accuracy: 0.7247
Epoch 5/50
100/100 [==============================] - 133s 1s/step - loss: 0.5642 - accuracy: 0.7271
Epoch 6/50
100/100 [==============================] - 131s 1s/step - loss: 0.5502 - accuracy: 0.7412
Epoch 7/50
100/100 [==============================] - 113s 1s/step - loss: 0.5510 - accuracy: 0.7362
Epoch 8/50
100/100 [==============================] - 113s 1s/step - loss: 0.5619 - accuracy: 0.7236
Epoch 9/50
100/100 [==============================] - 114s 1s/step - loss: 0.5511 - accuracy: 0.7309
Epoch 10/50
100/100 [============================

100/100 [==============================] - 115s 1s/step - loss: 0.5086 - accuracy: 0.7570
Epoch 27/50
100/100 [==============================] - 116s 1s/step - loss: 0.5148 - accuracy: 0.7608
Epoch 28/50
100/100 [==============================] - 116s 1s/step - loss: 0.5168 - accuracy: 0.7494
Epoch 29/50
100/100 [==============================] - 115s 1s/step - loss: 0.5172 - accuracy: 0.7539
Epoch 30/50
100/100 [==============================] - 115s 1s/step - loss: 0.4996 - accuracy: 0.7586
Epoch 31/50
100/100 [==============================] - 115s 1s/step - loss: 0.5148 - accuracy: 0.7592
Epoch 32/50
100/100 [==============================] - 115s 1s/step - loss: 0.5117 - accuracy: 0.7570
Epoch 33/50
100/100 [==============================] - 115s 1s/step - loss: 0.5134 - accuracy: 0.7498
Epoch 34/50
100/100 [==============================] - 116s 1s/step - loss: 0.4925 - accuracy: 0.7684
Epoch 35/50
100/100 [==============================] - 116s 1s/step - loss: 0.4882 - accuracy:

In [56]:
showScore()

Val loss: 0.5101664662361145
Val accuracy: 0.8787878751754761
Best score: 0.779036819934845


In [57]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min',
)

In [58]:
def get_model():    
    #Input shape = [width, height, color channels]
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    hidden=inputs
    
    
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    
    hidden = layers.Conv2D(238, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)    
    
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Flatten()(hidden)
    hidden = layers.Dense(64, activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    
    output = layers.Dense(3, activation='softmax')(hidden)     
    model = keras.Model(inputs,output)    
    return model

In [59]:
model = get_model()
model.compile(loss='categorical_crossentropy'
              , optimizer = keras.optimizers.Adam(), metrics='accuracy')

model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_19 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 52, 52, 238)       1373

In [60]:
history = model.fit(ds_train,
          batch_size = 100, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 253s 2s/step - loss: 1.0069 - accuracy: 0.5140 - val_loss: 0.7793 - val_accuracy: 0.6443
Epoch 2/50
130/130 [==============================] - 251s 2s/step - loss: 0.7514 - accuracy: 0.6433 - val_loss: 0.6550 - val_accuracy: 0.6942
Epoch 3/50
130/130 [==============================] - 252s 2s/step - loss: 0.6657 - accuracy: 0.6793 - val_loss: 0.5996 - val_accuracy: 0.7133
Epoch 4/50
130/130 [==============================] - 252s 2s/step - loss: 0.6487 - accuracy: 0.6848 - val_loss: 0.5760 - val_accuracy: 0.7162
Epoch 5/50
130/130 [==============================] - 251s 2s/step - loss: 0.6198 - accuracy: 0.6975 - val_loss: 0.5730 - val_accuracy: 0.7220
Epoch 6/50
130/130 [==============================] - 251s 2s/step - loss: 0.6127 - accuracy: 0.6982 - val_loss: 0.5783 - val_accuracy: 0.7191
Epoch 7/50
130/130 [==============================] - 251s 2s/step - loss: 0.6201 - accuracy: 0.7038 - val_loss: 0.5781 - val_accuracy: 0.7248

In [61]:
showScore()

Val loss: 0.3962772488594055
Val accuracy: 0.9696969985961914
Best score: 0.7397457361221313


In [62]:
def get_model():    
    #Input shape = [width, height, color channels]
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    hidden=inputs
    
    
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    
    hidden = layers.Conv2D(128, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)    
    
    hidden = layers.Conv2D(128, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Conv2D(64, padding='valid', kernel_size=(3,3),activation='relu')(hidden)
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Conv2D(32, padding='valid', kernel_size=(3,3),activation='relu')(hidden)   
    hidden = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    
    hidden = layers.Flatten()(hidden)
    hidden = layers.Dense(64, activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
    hidden = layers.Dropout(0.2)(hidden)    
    
    output = layers.Dense(3, activation='softmax')(hidden)     
    model = keras.Model(inputs,output)    
    return model

In [63]:
model = get_model()
model.compile(loss='categorical_crossentropy'
              , optimizer = keras.optimizers.Adam(), metrics='accuracy')

model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_24 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 52, 52, 128)       7385

In [64]:
history = model.fit(ds_train,
          batch_size = 50, 
          epochs = 50,
          validation_data=ds_val,
          callbacks=early_stopping,
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/50
130/130 [==============================] - 236s 2s/step - loss: 1.0646 - accuracy: 0.4824 - val_loss: 1.0533 - val_accuracy: 0.4851
Epoch 2/50
130/130 [==============================] - 231s 2s/step - loss: 1.0553 - accuracy: 0.4855 - val_loss: 1.0539 - val_accuracy: 0.4851
Epoch 3/50
130/130 [==============================] - 230s 2s/step - loss: 1.0528 - accuracy: 0.4855 - val_loss: 1.0514 - val_accuracy: 0.4851
Epoch 4/50
130/130 [==============================] - 229s 2s/step - loss: 1.0528 - accuracy: 0.4855 - val_loss: 1.0521 - val_accuracy: 0.4851
Epoch 5/50
130/130 [==============================] - 230s 2s/step - loss: 1.0528 - accuracy: 0.4855 - val_loss: 1.0504 - val_accuracy: 0.4851
Epoch 6/50
130/130 [==============================] - 229s 2s/step - loss: 1.0518 - accuracy: 0.4855 - val_loss: 1.0500 - val_accuracy: 0.4851
Epoch 7/50
130/130 [==============================] - 228s 2s/step - loss: 1.0514 - accuracy: 0.4852 - val_loss: 1.0460 - val_accuracy: 0.4851

In [65]:
showScore()

Val loss: 0.3985125422477722
Val accuracy: 0.9090909361839294
Best score: 0.7356680035591125
